In [5]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import json
import numpy as np
import os

In [19]:
diretorio = "../data/commits"

if not os.path.exists(diretorio):
    os.makedirs(diretorio)
    print(f"Diretório '{diretorio}' criado com sucesso.")
else:
    print(f"Diretório '{diretorio}' já existe.")

load_dotenv()

token1 = os.getenv('TOKEN1')
token2 = os.getenv('TOKEN2')
token3 = os.getenv('TOKEN3')
token4 = os.getenv('TOKEN4')

list_tokens = [token1, token2, token3, token4]

index_list_token = 0

df = pd.read_csv("../dataset/dataset_filtrado.csv", delimiter=';')

api_call = 1

def get_paginated_data(url):
    global api_call
    global index_list_token
    global list_tokens

    data = []
    while url:
        headers = {
            "Accept": "application/vnd.github+json",
            "Authorization": f"Bearer {list_tokens[index_list_token]}",
            "X-GitHub-Api-Version": "2022-11-28"
        }

        if api_call >= 4998:
            api_call = 1
            index_list_token += 1
        print(api_call, end=", ")
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            response_data = response.json()
            data.extend(parse_data(response_data))

            link_header = response.headers.get('Link')
            if link_header and 'rel="next"' in link_header:
                next_url = [url_part.split(";")[0].strip().strip("<>")
                            for url_part in link_header.split(",")
                            if 'rel="next"' in url_part]
                url = next_url[0] if next_url else None
            else:
                url = None
        else:
            print(f"Erro: {response.status_code}, {response.text}")
            break
        api_call = api_call + 1
    return data


def parse_data(response_data):
    if isinstance(response_data, list):
        return response_data
    elif isinstance(response_data, dict):
        namespace_key = list(response_data.keys())[0]
        return response_data[namespace_key]
    return []


def save_data_to_json(data, filename="data.json"):
    with open(filename, "w") as json_file:
        json.dump(data, json_file, indent=4)

Diretório '../data/commits' já existe.


In [20]:
for url in df['URL']:

    parts = url.split('https://github.com/')[1].split('/')

    owner = parts[0]
    repo = parts[1]

    url_final = f"https://api.github.com/repos/{owner}/{repo}/commits"

    print(url_final)

    data = get_paginated_data(url_final)

    filename = f"../data/commits/commits_{owner}_{repo}.json"

    save_data_to_json(data, filename)    

    print() # apenas para formatar a saida

https://api.github.com/repos/mirumee/saleor/commits
4997, trocou token
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 

KeyboardInterrupt: 